# Seasonal Decomposition - Solution Guide

This notebook contains the complete solutions for the seasonal decomposition exercises.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from utils.testing.forecasting_tests import check_seasonal_decomposition

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('husl')

In [ ]:
# Part 1: Understanding Components

# Generate sample data
dates = pd.date_range(start='2022-01-01', end='2023-12-31', freq='M')
base_trend = np.linspace(1000, 1500, len(dates))  # Upward trend
seasonal_pattern = 200 * np.sin(2 * np.pi * np.arange(len(dates)) / 12)  # Annual cycle
noise = np.random.normal(0, 50, len(dates))  # Random variations

sales_data = base_trend + seasonal_pattern + noise

# Create time series DataFrame
ts_data = pd.DataFrame({
    'sales': sales_data
}, index=dates)

# Perform decomposition
decomposition = seasonal_decompose(ts_data['sales'], period=12, model='additive')

# Plot components
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 10))

ts_data['sales'].plot(ax=ax1)
ax1.set_title('Original Time Series')

decomposition.trend.plot(ax=ax2)
ax2.set_title('Trend')

decomposition.seasonal.plot(ax=ax3)
ax3.set_title('Seasonal')

decomposition.resid.plot(ax=ax4)
ax4.set_title('Residual')

plt.tight_layout()
plt.show()

# Validate decomposition
check_seasonal_decomposition(decomposition)

In [ ]:
# Part 2: Additive vs Multiplicative Decomposition

# Generate data with multiplicative seasonality
multiplicative_seasonal = base_trend * (1 + 0.2 * np.sin(2 * np.pi * np.arange(len(dates)) / 12))

# Create DataFrame
mult_data = pd.DataFrame({
    'sales': multiplicative_seasonal
}, index=dates)

# Perform both decompositions
add_decomp = seasonal_decompose(mult_data['sales'], period=12, model='additive')
mult_decomp = seasonal_decompose(mult_data['sales'], period=12, model='multiplicative')

# Compare results
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Original data
mult_data['sales'].plot(ax=axes[0, 0])
axes[0, 0].set_title('Original Time Series')

# Trend comparison
add_decomp.trend.plot(ax=axes[1, 0], label='Additive')
mult_decomp.trend.plot(ax=axes[1, 0], label='Multiplicative')
axes[1, 0].set_title('Trend Comparison')
axes[1, 0].legend()

# Seasonal comparison
add_decomp.seasonal.plot(ax=axes[2, 0], label='Additive')
mult_decomp.seasonal.plot(ax=axes[2, 0], label='Multiplicative')
axes[2, 0].set_title('Seasonal Comparison')
axes[2, 0].legend()

# Residuals comparison
add_decomp.resid.plot(ax=axes[1, 1], label='Additive')
mult_decomp.resid.plot(ax=axes[1, 1], label='Multiplicative')
axes[1, 1].set_title('Residuals Comparison')
axes[1, 1].legend()

# Model selection metrics
add_rmse = np.sqrt(np.mean(add_decomp.resid.dropna()**2))
mult_rmse = np.sqrt(np.mean(mult_decomp.resid.dropna()**2))

axes[2, 1].text(0.1, 0.5, f'Additive RMSE: {add_rmse:.2f}\nMultiplicative RMSE: {mult_rmse:.2f}')
axes[2, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Part 3: Multiple Seasonal Patterns

# Generate hourly data
hourly_dates = pd.date_range(start='2023-01-01', end='2023-03-31', freq='H')
daily_pattern = 100 * np.sin(2 * np.pi * np.arange(len(hourly_dates)) / 24)
weekly_pattern = 50 * np.sin(2 * np.pi * np.arange(len(hourly_dates)) / (24 * 7))

hourly_sales = 1000 + daily_pattern + weekly_pattern + np.random.normal(0, 10, len(hourly_dates))

# Create DataFrame
hourly_data = pd.DataFrame({
    'sales': hourly_sales
}, index=hourly_dates)

# Analyze daily pattern
daily_decomp = seasonal_decompose(hourly_data['sales'], period=24, model='additive')

# Analyze weekly pattern (after removing daily pattern)
weekly_data = pd.DataFrame({
    'sales': daily_decomp.resid.fillna(method='bfill')
}, index=hourly_dates)
weekly_decomp = seasonal_decompose(weekly_data['sales'], period=24*7, model='additive')

# Plotting
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Original data
hourly_data['sales'].plot(ax=axes[0, 0])
axes[0, 0].set_title('Original Time Series')

# Daily pattern
daily_pattern_df = pd.DataFrame(daily_decomp.seasonal)
daily_pattern_df.iloc[:48].plot(ax=axes[1, 0])
axes[1, 0].set_title('Daily Pattern (48 hours)')

# Weekly pattern
weekly_pattern_df = pd.DataFrame(weekly_decomp.seasonal)
weekly_pattern_df.iloc[:24*7].plot(ax=axes[2, 0])
axes[2, 0].set_title('Weekly Pattern')

# Combined effects
combined_seasonal = daily_decomp.seasonal + weekly_decomp.seasonal
pd.DataFrame(combined_seasonal).iloc[:24*7].plot(ax=axes[1, 1])
axes[1, 1].set_title('Combined Seasonal Effects')

# Pattern strength comparison
daily_strength = np.std(daily_decomp.seasonal)
weekly_strength = np.std(weekly_decomp.seasonal)
axes[2, 1].text(0.1, 0.5, 
                f'Pattern Strength:\nDaily: {daily_strength:.2f}\nWeekly: {weekly_strength:.2f}')
axes[2, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Part 4: Seasonal Adjustment

# Function to perform seasonal adjustment
def adjust_seasonal(data, period):
    decomp = seasonal_decompose(data, period=period, model='additive')
    adjusted = data - decomp.seasonal
    return adjusted, decomp

# Adjust original monthly sales data
adjusted_sales, original_decomp = adjust_seasonal(ts_data['sales'], period=12)

# Create comparison plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Original vs Adjusted
ts_data['sales'].plot(ax=ax1, label='Original')
adjusted_sales.plot(ax=ax1, label='Seasonally Adjusted')
ax1.set_title('Original vs Seasonally Adjusted Sales')
ax1.legend()

# Trend analysis
from scipy import stats

# Linear regression on adjusted data
X = np.arange(len(adjusted_sales)).reshape(-1, 1)
y = adjusted_sales.values.reshape(-1, 1)
reg = stats.linregress(X.flatten(), y.flatten())

# Plot trend
trend_line = reg.slope * X + reg.intercept
adjusted_sales.plot(ax=ax2, label='Adjusted Data')
ax2.plot(adjusted_sales.index, trend_line, 'r--', label=f'Trend (slope={reg.slope:.2f})')
ax2.set_title('Trend Analysis on Adjusted Data')
ax2.legend()

plt.tight_layout()
plt.show()

# Print trend statistics
print(f'Trend Statistics:')
print(f'Slope: {reg.slope:.4f}')
print(f'R-squared: {reg.rvalue**2:.4f}')
print(f'P-value: {reg.pvalue:.4f}')